In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board
import matplotlib.pyplot as plt

import numpy as np

In [2]:
training_generator = DataLoader(128, "train")
val_generator = DataLoader(128, "val", shuffle=False)

In [3]:
lossHistory = []
valHistory = []
class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['loss'])
    if batch and batch % 1000 == 0:
      pred = self.model.predict(val_generator)
      loss = np.abs(pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*128])
      loss = np.sum(loss) / len(loss)
      valHistory.append(loss)

      plt.plot(valHistory)
      plt.savefig("val.png", dpi=400)
      plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=400)
      plt.clf()
      
      print(" val_loss : {}".format(loss))

      self.model.save("models/model-{}".format(len(valHistory)))

In [4]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=2)

In [8]:
input1 = Input((64*6,), sparse=True)
input2 = Input((64*6,), sparse=True)
accumulator = Dense(64, clipped_relu)
layers = concatenate([accumulator(input1), accumulator(input2)])
layers = Dense(32, clipped_relu)(layers)
layers = Dense(32, clipped_relu)(layers)
layers = Dense(1, 'sigmoid')(layers)
model = Model(inputs=[input1, input2], outputs=layers)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [6]:
model = tf.keras.models.load_model("production/")
# model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), loss='mse', metrics=['mae'])

In [ ]:
model.fit(training_generator, validation_data=val_generator, epochs=2, callbacks=[NnueCallbacks()])

In [9]:
ids = get_halfkp_indeicies(Board("r3k1nr/pp2ppbp/n2p2p1/q1p5/3N1P2/1P2P3/PBPP3P/RN1K1B1R w kq - 1 9"))

ids = ids[ids > 0]
h1 = ids[ids >= 384] - 384
h2 = ids[ids < 384]

x = (np.zeros((2, 384), bool), np.zeros((2, 384), bool))

np.add.at(x[0][0], h1, h1.astype(bool))
np.add.at(x[1][0], h2, h2.astype(bool))

np.add.at(x[0][1], h1, h1.astype(bool))
np.add.at(x[1][1], h2, h2.astype(bool))

pred = model.predict(x)
print(pred[0])

ValueError: in user code:

    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 199, in assert_input_compat

In [ ]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

In [18]:
import time
# model.save("models/model-{}".format(int(time.time())))
model.save("production/")

INFO:tensorflow:Assets written to: production/assets


INFO:tensorflow:Assets written to: production/assets
